# Q1
## Use of PERMNO vs. TICKERS:

PERMNO is a unique identifier assigned by the Center for Research in Security Prices (CRSP) to each security in the market. It is used in academic and research settings because it provides a consistent way to track securities over time, regardless of changes in ticker symbols, company names, or corporate actions such as mergers and acquisitions. This consistency is crucial for historical analysis over long periods, as it allows for the unambiguous identification of securities.

TICKERS, on the other hand, are symbols assigned to securities traded on public exchanges. These can change over time due to various reasons, such as rebranding, mergers, or moving between exchanges, which can create confusion in longitudinal studies.

The choice of PERMNO over tickers for the dataset is likely aimed at ensuring consistency and accuracy in tracking the performance and characteristics of securities over time, especially in a dataset that spans several years.

# Q2
## Discrepancies between One-Month Price Change and PRC Column:

a. This discrepancy is not necessarily a data error. The PRC column typically represents the end-of-month stock price, while the one-month percent change in price is calculated based on the price change from the previous month to the current month. Several factors can cause these numbers to diverge, including stock splits, dividends, and other corporate actions that affect the stock price but are not reflected in the simple price listed.

b. For some companies, these columns may always be equal due to the absence of such corporate actions during the period in question or because the stock's price changes purely reflect market trading without external adjustments. This uniformity could also be seen in companies with stable dividend policies or those that did not undergo any stock splits or similar adjustments during the period.

# Q3
## Variability in the Number of Companies:

The number of companies in the S&P 500 index does not always precisely equal 500 due to several factors, including mergers, acquisitions, bankruptcies, and the addition or removal of companies based on the market capitalization criteria set by the index. These events can cause the number of constituents to fluctuate temporarily until adjustments are made.

It's not a mistake but reflects the dynamic nature of the stock market and the index's composition adjustments to maintain its representation of the U.S. economy's leading companies.

# Q4

## 303 companies are present over the entire sample 
## 761 unique companies are in the sample

In [17]:
import pandas as pd

file_path = 'sp500raw.xlsx'
sp500_data = pd.read_excel(file_path)
sp500_data.drop(columns=['Unnamed: 0'], inplace =True)

display(sp500_data.head())

date_counts = sp500_data.groupby('permno').size()
full_presence_companies = date_counts[date_counts == sp500_data['date'].nunique()]

unique_companies = sp500_data['permno'].nunique()

(full_presence_companies.count(), unique_companies)

,permno,date,price,shrout,prc,mcap
0,43449,2011-01-31,73.67,1056507,-0.040255,77832870.69
1,13688,2011-01-31,46.28,392066,-0.032609,18144814.48
2,90071,2011-01-31,20.75,247197,0.061924,5129337.75
3,86580,2011-01-31,23.92,588556,0.553247,14078259.52
4,10225,2011-01-31,61.68,152581,0.023734,9411196.08


(303, 761)

# Q5

